In [22]:
import re
import string
import nltk
import random

from nltk import FreqDist
from typing import Tuple, Callable
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords

In [6]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')

In [7]:
tweet_tokens[0]

['#FollowFriday',
 '@France_Inte',
 '@PKuchly57',
 '@Milipol_Paris',
 'for',
 'being',
 'top',
 'engaged',
 'members',
 'in',
 'my',
 'community',
 'this',
 'week',
 ':)']

### Normalizing data

In [8]:
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

#### Removing noise

In [9]:
def lemmatize_sentence(normalization_func: Callable, tokens, stop_words: Tuple = ()):
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        word_ = normalization_func(word, tag)
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
    
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(word_, pos)
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            lemmatized_sentence.append(token.lower())
            
    return lemmatized_sentence


def remove_noise(token: str, tag: str):
    token_ = re.sub(
            r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|"\"(?:%[0-9a-fA-F][0-9a-fA-F]))+',
            '',
            token
    )
    token_ = re.sub(r'(@[A-Za-z0-9_]+)', '', token_)
    
    return token_

In [10]:
stop_words = stopwords.words('english')
print(lemmatize_sentence(remove_noise, tweet_tokens[0], stop_words))

['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']


In [11]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')
positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(
        lemmatize_sentence(
            remove_noise,
            tokens,
            stop_words
        )
    )

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(
        lemmatize_sentence(
            remove_noise,
            tokens,
            stop_words
        )
    )

### Determining word density

In [12]:
def get_all_words(cleaned_token_list):
    for tokens in cleaned_token_list:
        for token in tokens:
            yield token

In [13]:
all_pos_words = get_all_words(positive_cleaned_tokens_list)

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


In [14]:
all_neg_words = get_all_words(negative_cleaned_tokens_list)

freq_dist_neg = FreqDist(all_neg_words)
print(freq_dist_neg.most_common(10))

[(':(', 4585), (':-(', 501), ("i'm", 343), ('...', 332), ('get', 325), ('miss', 291), ('go', 275), ('please', 275), ('want', 246), ('like', 218)]


### Preparing data to the model

In [15]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [21]:
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

In [23]:
positive_dataset = [(tweet_dict, 'Positive') for tweet_dict in positive_tokens_for_model]
negative_dataset = [(tweet_dict, 'Negative') for tweet_dict in negative_tokens_for_model]
dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

### Creating and testing the model

In [26]:
from nltk import classify
from nltk import NaiveBayesClassifier


classifier = NaiveBayesClassifier.train(train_data)
print(f'Accuracy: {classify.accuracy(classifier, test_data)}')
print(classifier.show_most_informative_features(10))

Accuracy: 0.996
Most Informative Features
                      :( = True           Negati : Positi =   2093.8 : 1.0
                      :) = True           Positi : Negati =    983.6 : 1.0
                follower = True           Positi : Negati =     38.1 : 1.0
                     sad = True           Negati : Positi =     20.8 : 1.0
                     x15 = True           Negati : Positi =     18.1 : 1.0
                followed = True           Negati : Positi =     15.3 : 1.0
               community = True           Positi : Negati =     15.3 : 1.0
                 welcome = True           Positi : Negati =     14.9 : 1.0
                    glad = True           Positi : Negati =     14.3 : 1.0
                     via = True           Positi : Negati =     14.2 : 1.0
None


In [30]:
from nltk.tokenize import word_tokenize

custom_negative_tweet = 'I ordered just once from TerribleCo, they screwed up, never used the app again.'
custom_negative_tokens = lemmatize_sentence(
    remove_noise,
    word_tokenize(custom_negative_tweet)
)
print(classifier.classify(dict([token, True] for token in custom_negative_tokens)))

Negative


In [31]:
custom_positive_tweet = 'Congrats #SportStar on your 7th best goal from last season winning goal of the year :) #Baller #Topbin #oneofmanyworldies'
custom_positive_tokens = lemmatize_sentence(
    remove_noise,
    word_tokenize(custom_positive_tweet)
)
print(classifier.classify(dict([token, True] for token in custom_positive_tokens)))

Positive


In [33]:
custom_sarcasm_tweet = 'Thank you for sending my baggage to CityX and flying me to CityY at the same time. Brilliant service. #thanksGenericAirline'
custom_sarcasm_tokens = lemmatize_sentence(
    remove_noise,
    word_tokenize(custom_sarcasm_tweet)
)
print(classifier.classify(dict([token, True] for token in custom_sarcasm_tokens)))

Positive
